In [1]:
import ee
#ee.Authenticate()
ee.Initialize()

In [2]:
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob
import geemap

# Load cities data & choose city

In [7]:
## Load metro area boundaries
cities = gpd.read_file("data/Smart_Surfaces_metro_areas/smart_surfaces_urban_areas.shp")
print(cities)

    OBJECTID UACE10 GEOID10                               NAME10  \
0        558  42346   42346                      Jacksonville_FL   
1        608  80389   80389                           Seattle_WA   
2        684  15670   15670                     Charlotte_NC__SC   
3        714  71317   71317                      Portland_OR__WA   
4        816  56602   56602                             Miami_FL   
5        990  58600   58600                        Montgomery_AL   
6       1101  23824   23824                           Detroit_MI   
7       1584  09271   09271                    Boston_MA__NH__RI   
8       1625  77770   77770                      St_Louis_MO__IL   
9       1666  77068   77068                        Sacramento_CA   
10      2312  63217   63217          New_York__Newark_NY__NJ__CT   
11      2537  69076   69076          Philadelphia_PA__NJ__DE__MD   
12      2709  56116   56116                   Memphis_TN__MS__AR   
13      2834  22042   22042     Dallas__Fort_Wor

In [9]:
## Choose city of interest, use the GEOID
city = '62677'

## Get city geometry
city_geom = cities.loc[cities['GEOID10'] == city]
print(city_geom)

    OBJECTID UACE10 GEOID10          NAME10                      NAMELSAD10  \
18      3377  62677   62677  New_Orleans_LA  New Orleans, LA Urbanized Area   

   LSAD10 MTFCC10 UATYP10 FUNCSTAT10      ALAND10    AWATER10   INTPTLAT10  \
18     75   G3500       U          S  651216426.0  44655876.0  +29.9568739   

      INTPTLON10  UA_ID     Shape_Leng    Shape_Area             UHI_NAME  \
18  -090.1414405  62677  599090.322152  6.958725e+08  3377_New_Orleans_LA   

                                             geometry  
18  MULTIPOLYGON (((-90.05282 30.03435, -90.05221 ...  


# OSM

In [25]:
# query OSM
## create bounding box
bb = city_geom.buffer(100).bounds

print(city_geom.crs)

## return the list of coordinates
# listCoords = ee.Array.cat(bb.coordinates(), 1); 

# ##Casting it to an array makes it possible to slice out the x and y coordinates:
# ##get the X-coordinates
# xCoords = listCoords.slice(1, 0, 1); #print('xCoords', xCoords.getInfo());
# yCoords = listCoords.slice(1, 1, 2); #print('yCoords', yCoords.getInfo());

# ## Reducing each array reveals then the min and max values:
# ## reduce the arrays to find the max (or min) value
# West = ee.Number(xCoords.reduce('min', [0]).get([0,0])).getInfo(); 
# East = ee.Number(xCoords.reduce('max', [0]).get([0,0])).getInfo(); 
# North = ee.Number(yCoords.reduce('min', [0]).get([0,0])).getInfo(); 
# South = ee.Number(yCoords.reduce('max', [0]).get([0,0])).getInfo(); 

EPSG:4326


/var/folders/jn/2s8m6y5x09z9ysrcz_t9z2qw0000gq/T/ipykernel_7864/556780835.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bb = city_geom.buffer(100).bounds
